> **This notebook uses a langchain `create_sql_agent` to query a SQL database.**
> - **Thought:** Use this agent for various purposes: querying data, generating reports, authoring documents, and maybe updating data
> - **Observations:**
>   - LangChain `SQLDatabase` can only work with the default schema, until [this issue](https://github.com/langchain-ai/langchain/issues/3036) is resolved.
>   - The agent queries the available tables and its schema first.
>   - It will generate sql scripts to try, and keep retrying if the sql script fails -- until it reaches the model token limit.
>   - From trials, it requires a lot of trials to get a query to work. And it can fail due to complex joins. 
> - Sample Data: [Northwind Traders](https://www.dofactory.com/sql/sample-database), bacause AdventureWorks uses a `SalesLT` schema

## Environment Setup

In [1]:
import os
from dotenv import load_dotenv

# Set environment variables in the .env file.
load_dotenv()

COMPLETION_MODEL = os.environ["OPENAI_COMPLETION_MODEL"]
COMPLETION_DEPLOYMENT = os.environ["OPENAI_COMPLETION_DEPLOYMENT"]
CHAT_MODEL = os.environ["OPENAI_CHAT_MODEL"]
CHAT_DEPLOYMENT = os.environ["OPENAI_CHAT_DEPLOYMENT"]
OPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"]

SQL_DB_USER = os.environ["SQL_DB_USER"]
SQL_DB_PASSWORD = os.environ["SQL_DB_PASSWORD"]
SQL_DB_SERVER_NAME = os.environ["SQL_DB_SERVER_NAME"]
SQL_DB_NAME = os.environ["SQL_DB_NAME"]
SQL_CONNECTIONSTRING_FORMAT = os.environ["SQL_CONNECTIONSTRING_FORMAT"]

In [2]:
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

llm = AzureOpenAI(
    model_name=COMPLETION_MODEL,
    deployment_name=COMPLETION_DEPLOYMENT,
    #temperature=0.3,
    verbose=True
)

chat = AzureChatOpenAI(
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION,
    #temperature=0.3,
    verbose=True
)

## Code

In [8]:
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

connection_string = SQL_CONNECTIONSTRING_FORMAT.format(
    database_user=SQL_DB_USER,
    database_password=SQL_DB_PASSWORD,
    database_server=SQL_DB_SERVER_NAME,
    database_db=SQL_DB_NAME)

db = SQLDatabase.from_uri(connection_string)


### Attempt 1: Pure SQLDatabaseToolkit

In [9]:
from langchain.agents import create_sql_agent

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
sqlagent = create_sql_agent(
    llm=chat,
    toolkit=toolkit,
    verbose=True
)

In [16]:
# Tried adding the following into the prompt.
# However, the SQL Agent will still run sql_db_list_tables and generate based on this information

# metaprompt = """You are an AI assistant that helps me answer questions about data in a database.
# The database contains the following tables, columns and datatypes:
# Customer
# - Id (int)
# - FirstName (string)
# - LastName (string)
# - City (string)
# - Country (string)
# - Phone (string)

# Order
# - Id (int)
# - OrderDate (datetime)
# - OrderNumber (int)
# - CustomerId (int), joins to Customer.Id
# - TotalAmount (decimal)

# OrderItem
# - Id (int)
# - OrderId (int), joins to Order.Id
# - ProductId (int),
# - UnitPlace (decimal),
# - Quantity (int)

# Supplier
# - Id (int),
# - CompanyName (string),
# - ContactName (string),
# - ContactTitle (string),
# - City (string),
# - Country (string),
# - Phone (string),
# - Fax (string)

# Product
# - Id (int),
# - ProductName (string),
# - SupplierId (int), joins to Supplier.Id,
# - UnitPrice (decimal),
# - Package (string),
# - IsDiscontinued (bit)
#
# Question: """

metaprompt = """You are an AI assistant that helps me answer questions about data in a database.
Question: """

In [24]:
# Attempt Log
# 1. Prompt: list top 5 customers by highest total order amount
#    Output: SELECT TOP 5 Customer.FirstName, Customer.LastName, [Order].TotalAmount FROM Customer JOIN [Order] ON Customer.Id = [Order].CustomerId ORDER BY [Order].TotalAmount DESC
#    Comment: Should be SUM(TotalAmmount), GroupBY Customer.FirstName, Customer.LastName
# 2. Prompt: list top 5 customers by sum of TotalAmount grouped by customer name
#    Output: SELECT TOP 5 WITH TIES c.FirstName + ' ' + c.LastName AS CustomerName, SUM(o.TotalAmount) AS TotalAmount
#              FROM Customer c
#              JOIN [Order] o ON c.Id = o.CustomerId
#              GROUP BY c.FirstName, c.LastName
#              ORDER BY TotalAmount DESC
#    Comment: Correct but query language is not very natural
# 3. Prompt: list top 5 customers by highest UnitPrice*Quantity
#    Output: SELECT TOP 5 c.FirstName, c.LastName, SUM(oi.UnitPrice * oi.Quantity) AS TotalCost 
#               FROM Customer c 
#               JOIN OrderItem oi ON c.Id = oi.OrderId 
#               GROUP BY c.FirstName, c.LastName 
#               ORDER BY TotalCost DESC
#    Comment: Wrong Join. OrderItem should be joined to Order first, before getting the Customer Id.

sqlagent.run(metaprompt + "list top 5 customers by highest UnitPrice*Quantity")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Customer, Order, OrderItem, Product, Supplier
Thought:I can query the OrderItem table to get the UnitPrice and Quantity of each item. Then I can join it with the Customer table to get the customer information.
Action: sql_db_schema
Action Input: "Customer, OrderItem"
Observation: 
CREATE TABLE [Customer] (
	[Id] INTEGER NOT NULL IDENTITY(1,1), 
	[FirstName] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[LastName] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[City] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Country] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Phone] NVARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK_CUSTOMER] PRIMARY KEY ([Id])
)

/*
3 rows from Customer table:
Id	FirstName	LastName	City	Country	Phone
1	Maria	Anders	Berlin	Germany	030-0074321
2	Ana	Trujillo	México D.F.	Mexico	(5) 555-4729
3

'Liz Nixon, Helvetius Nagy, Guillermo Fernández, Roland Mendel, and Christina Berglund are the top 5 customers by highest UnitPrice*Quantity.'

### Attempt 2: Two tools: SQLDatabaseToolkit and Math

In [21]:
toolkit2 = SQLDatabaseToolkit(db=db, llm=llm)
sqlagent2 = create_sql_agent(
    llm=chat,
    toolkit=toolkit2,
    verbose=True
)

from langchain.agents import load_tools
tool_math = load_tools(["llm-math"], llm=llm)
sqlagent2.tools = sqlagent2.tools + tool_math


In [23]:
sqlagent2.run(metaprompt + "list top 5 customers by highest UnitPrice*Quantity")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Customer, Order, OrderItem, Product, Supplier
Thought:I can query the OrderItem table to get the UnitPrice and Quantity for each customer. Then I can join the Customer table to get the customer details.
Action: sql_db_schema
Action Input: "Customer, OrderItem"
Observation: 
CREATE TABLE [Customer] (
	[Id] INTEGER NOT NULL IDENTITY(1,1), 
	[FirstName] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[LastName] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[City] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Country] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Phone] NVARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK_CUSTOMER] PRIMARY KEY ([Id])
)

/*
3 rows from Customer table:
Id	FirstName	LastName	City	Country	Phone
1	Maria	Anders	Berlin	Germany	030-0074321
2	Ana	Trujillo	México D.F.	Mexico	(5) 555-4729
3	Antoni

'Liz Nixon, Helvetius Nagy, Guillermo Fernández, Roland Mendel, and Christina Berglund.'